# Data prediction

## Imports

In [315]:
from pandas import DataFrame
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

from datetime import datetime
from datetime import timedelta

## Data preprocessing

In [316]:
df = pd.read_csv('/data/DEvideos.csv',
    low_memory=False)

df['trending_date'] = df.apply(lambda row: datetime.strptime(row['trending_date'], '%y.%d.%m'), axis=1)
df['publish_time'] = df.apply(lambda row: datetime.strptime(row['publish_time'], '%Y-%m-%dT%H:%M:%S.000Z'), axis=1)
df['days_until_trending'] = df.apply(lambda row: ((row['trending_date'] - row['publish_time']).days + 1), axis=1)

df['tags_count'] = df.apply(lambda row: len(row['tags'].split('|')), axis=1)
df['publish_hour'] = df['publish_time'].map(lambda x: x.hour)
df['publish_month'] = df['publish_time'].map(lambda x: x.month)
df['publish_year'] = df['publish_time'].map(lambda x: x.year)
df['publish_day_of_month'] = df['publish_time'].map(lambda x: x.day)
df['publish_weekday'] = df['publish_time'].map(lambda x: x.weekday()) # 0: Monday, 6: Sunday
df['trending_weekday'] = df['trending_date'].map(lambda x: x.weekday()) # 0: Monday, 6: Sunday

df['like_dislike_ratio'] = df.apply(lambda row: row['likes'] / (row['dislikes'] + 1), axis=1)
df['like_view_ratio'] = df.apply(lambda row: row['likes'] / (row['views'] + 1), axis=1)

df['ratings'] = df['likes'] + df['dislikes']
df['likes_per_rating'] = df['likes'] / df['ratings']
df['ratings_per_view'] = df['ratings'] / df['views']
df['comments_per_view'] = df['comment_count'] / df['views']

# Using int instead of cat
def assign_target_category(row):
    if row['days_until_trending'] == 0: 
        return 0
    elif row['days_until_trending'] == 1:
        return 1
    elif row['days_until_trending'] == 2:
        return 2
    elif row['days_until_trending'] <= 5:
        return 3
    else:
        return 6

df['target_category'] = df.apply(assign_target_category, axis=1)

N = len(df)
dropColumns = ['video_id', 'title', 'tags', 'thumbnail_link', 'description']
for column in df.columns:
    numberOfUniqueValues = df[column].nunique()
    if numberOfUniqueValues < 2:
        dropColumns.append(column)
    elif df[column].dtype == 'object' and numberOfUniqueValues > N * 0.9:
        dropColumns.append(column)
    elif df[column].isna().sum() / N > 0.95:
        dropColumns.append(column)
        
df.drop(columns=dropColumns, inplace=True)

df['channel_title'] = df['channel_title'].astype('category')

## Encode features

In [317]:
x_df = DataFrame(index=df.index)
features = ['publish_weekday', 'publish_month']
for feature in features:
    feature_data = df[feature]
    if df[feature].dtype.name == 'category':
        x_label_encoder = preprocessing.LabelEncoder()
        x_label_encoder.fit(feature_data.astype(str))
        x_df[feature] = x_label_encoder.transform(feature_data)
    elif df[feature].dtype.name == 'datetime64[ns]':
        x_df[feature] = feature_data.to_seconds()
    elif df[feature].dtype.name == 'bool':
        x_df[feature] = int(feature_data)
    else:
        x_df[feature] = feature_data

x = np.reshape(x_df, (-1, len(x_df.columns)))

## Encode prediction target

In [318]:
target = df['target_category'].astype(str)
y_label_encoder = preprocessing.LabelEncoder()
y_label_encoder.fit(target)
y = y_label_encoder.transform(target)

## Create train and test datasubset

In [319]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.40)

## Declare classifiers

```
classifiers[:][0]: Name
classifiers[:][1]: Classifier object
classifiers[:][2]: Prediction
```

In [320]:
classifiers = []

classifiers.append(['Decision Tree', DecisionTreeClassifier()])
classifiers.append(['K Nearest Neighbor', KNeighborsClassifier()])
classifiers.append(['Random Forest', RandomForestClassifier()])
classifiers.append(['XG Boost', XGBClassifier(use_label_encoder=False)])

classifiers.append(['K Nearest Neighbor (optimised)', KNeighborsClassifier(n_neighbors = 1000)])
classifiers.append(['Random Forest (optimised)', RandomForestClassifier(n_estimators=10, max_depth=2, min_samples_split=5)])
classifiers.append(['XG Boost (optimised)', XGBClassifier(n_estimators=50, max_depth=2, use_label_encoder=False)])

## Train model

In [321]:
for classifier in classifiers:
    print('\nTraining - ' + classifier[0])
    %time classifier[1].fit(x_train, y_train)


Training - Decision Tree
CPU times: user 15.4 ms, sys: 5 ms, total: 20.4 ms
Wall time: 16.4 ms

Training - K Nearest Neighbor
CPU times: user 59.4 ms, sys: 1.01 ms, total: 60.4 ms
Wall time: 59.9 ms

Training - Random Forest
CPU times: user 543 ms, sys: 0 ns, total: 543 ms
Wall time: 542 ms

Training - XG Boost
CPU times: user 6.41 s, sys: 203 ms, total: 6.62 s
Wall time: 1.66 s

Training - K Nearest Neighbor (optimised)
CPU times: user 200 ms, sys: 12 ms, total: 212 ms
Wall time: 56.1 ms

Training - Random Forest (optimised)
CPU times: user 41.7 ms, sys: 1.01 ms, total: 42.7 ms
Wall time: 42.3 ms

Training - XG Boost (optimised)
CPU times: user 1.27 s, sys: 39 ms, total: 1.31 s
Wall time: 329 ms


## Predict test data

In [322]:
for classifier in classifiers:
    print('\nPredicting - ' + classifier[0])
    %time classifier.append(classifier[1].predict(x_test))


Predicting - Decision Tree
CPU times: user 24.7 ms, sys: 0 ns, total: 24.7 ms
Wall time: 8.12 ms

Predicting - K Nearest Neighbor
CPU times: user 846 ms, sys: 26 ms, total: 872 ms
Wall time: 621 ms

Predicting - Random Forest
CPU times: user 210 ms, sys: 1e+03 ns, total: 210 ms
Wall time: 209 ms

Predicting - XG Boost
CPU times: user 293 ms, sys: 5.02 ms, total: 298 ms
Wall time: 76 ms

Predicting - K Nearest Neighbor (optimised)
CPU times: user 8.37 s, sys: 245 ms, total: 8.62 s
Wall time: 8.02 s

Predicting - Random Forest (optimised)
CPU times: user 19.2 ms, sys: 8 µs, total: 19.2 ms
Wall time: 18.6 ms

Predicting - XG Boost (optimised)
CPU times: user 55.9 ms, sys: 1.92 ms, total: 57.8 ms
Wall time: 15.2 ms


## Calculate accuracy

In [323]:
for classifier in classifiers:
    print('\nAccuracy - ' + classifier[0])
    print(accuracy_score(y_test, classifier[2]))


Accuracy - Decision Tree
0.6256733594515181

Accuracy - K Nearest Neighbor
0.5806194906953966

Accuracy - Random Forest
0.625550930460333

Accuracy - XG Boost
0.6258570029382958

Accuracy - K Nearest Neighbor (optimised)
0.6254285014691479

Accuracy - Random Forest (optimised)
0.6254285014691479

Accuracy - XG Boost (optimised)
0.6254285014691479


## Accuracy of baseline classification

In [324]:
len(df[df['target_category'] == 1]) / len(df)

0.6237512242899118